# Active Qubit Reset on QCS

In this notebook, we will walk through how to use **Active Qubit Reset** to drastically decrease the amount of time it takes to run a job on the QPU. Although we use a toy example in this notebook, the principles here can be extended to rapidly iterate through real-world applications such as optimizing a variational quantum algorithm (as we will do in the **Max-Cut QAOA** notebook).

In [ ]:
import itertools
import time
from typing import List

from pyquil import get_qc, Program
from pyquil.gates import CNOT, H, MEASURE, RESET

## Get Started

Before running on the QPU, users must book a block of time on`Aspen-8`using the QCS command-line interface. Once your QPU time has started (which we call being "QPU-engaged"), you must then set up the `QuantumComputer` object associated with the booked lattice, which we do in the following cell.

In [ ]:
lattice = 'Aspen-8'  # edit as necessary
qpu = get_qc(lattice)
qubits = qpu.device.qubits()[-3:]  # edit as necessary
print(f'All qubits on {lattice}: {qpu.device.qubits()}')
print(f'\nSelected qubits: {qubits}')

## Build GHZ Program

We begin by putting the first qubit in the superposition state |+⟩ by using the [Hadamard](https://en.wikipedia.org/wiki/Quantum_logic_gate#Hadamard_(H)_gate) gate. Then, we produce our [Greenberger–Horne–Zeilinger](https://en.wikipedia.org/wiki/Greenberger%E2%80%93Horne%E2%80%93Zeilinger_state) (GHZ) state (which looks like |000⟩ + |111⟩ for 3 qubits), by entangling all the qubits successively using [Controlled-NOT](https://en.wikipedia.org/wiki/Controlled_NOT_gate) (`CNOT`) gates. As in the **Parametric Compilation** notebook, we also declare our readout memory "ro", and measure each qubit into a readout register.

In [ ]:
def ghz_program(qubits: List[int]) -> Program:
    program = Program()
    program.inst(H(qubits[0]))
    for i in range(len(qubits) - 1):
        program.inst(CNOT(qubits[i], qubits[i + 1]))
    ro = program.declare('ro', 'BIT', len(qubits))
    program.inst([MEASURE(qubit, ro[idx]) for idx, qubit in enumerate(qubits)])
    return program

## Enable Active Reset

We create two GHZ state programs, one with an initial `RESET` command, and one without. The `RESET` directive enables active qubit reset for all the measured qubits in the program. We then set the number of shots to take for each Quil program, and compile each into instrument binaries.

In [ ]:
program = Program()
program.inst(ghz_program(qubits))
program.wrap_in_numshots_loop(10_000)
binary = qpu.compile(program)

program_reset = Program(RESET(), ghz_program(qubits))
program_reset.wrap_in_numshots_loop(10_000)
binary_reset = qpu.compile(program_reset)

## Compare Execution Time

We run each binary on the QPU, comparing the total run time, to see a drastic speed increase with active qubit reset. For our Aspen-8 system, we are able to achieve an order of magnitude reduction in reset time between passive and active reset (~100μs vs. ~10μs). However, there are additional components beyond reset time that make up the total execution time, which is why we only see a ~5x overall improvement.

In [ ]:
start = time.time()
results = qpu.run(binary)
total = time.time() - start
print(f'Execution time without active reset: {total:.3f} s')

start_reset = time.time()
results_reset = qpu.run(binary_reset)
total_reset = time.time() - start_reset
print(f'\nExecution time with active reset: {total_reset:.3f} s')

## Compare Execution Quality

We compare the bitstring counts to see that there is no material performance hit for using active qubit reset. As we see in both count dictionaries, the `000` and `111` bitstrings are the most prevalent, as expected. However, we can see that the bitstring counts aren't perfect, which we can attribute to gate infidelity and decoherence.

In [ ]:
counts = {bit_tuple: 0 for bit_tuple in itertools.product((0, 1), repeat=3)}
for shot_result in results:
    bit_tuple = tuple(shot_result)
    counts[bit_tuple] += 1  
print(f'Measurement results without active reset:')
for bit_tuple, count in counts.items():
    print(bit_tuple, count)

counts_reset = {bit_tuple: 0 for bit_tuple in itertools.product((0, 1), repeat=3)}
for shot_result in results_reset:
    bit_tuple = tuple(shot_result)
    counts_reset[bit_tuple] += 1
print(f'\nMeasurement results with active reset:')
for bit_tuple, count in counts_reset.items():
    print(bit_tuple, count)